In [ ]:
# 3D Deepfake Generation and Detection

# Authored by Hichem Felouat
# Email            : hichemfel@nii.ac.jp
# GitHub Repository: https://github.com/hichemfelouat/3DDGD.git



In [1]:
!nvidia-smi

Fri Jun 13 04:51:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### **Clone Repository:**

In [ ]:
!git clone https://github.com/hichemfelouat/3DDGD.git

In [5]:
%cd 3DDGD

/content/3DDGD


In [7]:
!unzip /content/3DDGD/weights.zip

Archive:  /content/3DDGD/weights.zip
  inflating: weights/Tabtransformer/Tabtransformer_G0.pth  
  inflating: weights/Tabtransformer/Tabtransformer_G1.pth  
  inflating: weights/Tabtransformer/Tabtransformer_G2.pth  
  inflating: weights/Mesh_MLP_MHA/Mesh_MLP_MHA_G1.pth  
  inflating: weights/Mesh_MLP_MHA/Mesh_MLP_MHA_G2.pth  
  inflating: weights/Mesh_MLP_MHA/Mesh_MLP_MHA_G0.pth  


### **Requirements:**

In [ ]:
"""
!pip install pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=12.1 -c pytorch -c nvidia

!pip install trimesh
!pip install open3d
!pip install libigl
!pip install robust-Laplacian

!pip install scipy==1.11.3
!pip install shapely==2.0.3
!pip install pyglet==1.5.27
!pip install mediapipe==0.10.11
!pip install timm==1.0.9
"""

'\n!pip install pytorch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 pytorch-cuda=12.1 -c pytorch -c nvidia\n\n!pip install trimesh\n!pip install open3d\n!pip install libigl\n!pip install robust-Laplacian\n\n!pip install scipy==1.11.3\n!pip install shapely==2.0.3\n!pip install pyglet==1.5.27\n!pip install mediapipe==0.10.11\n!pip install timm==1.0.9\n'

In [4]:
!pip install trimesh
!pip install open3d
!pip install libigl
!pip install robust-Laplacian
#!pip install scipy

# For face cropping
!pip install scipy==1.11.3
!pip install shapely==2.0.3
!pip install pyglet==1.5.27
!pip install mediapipe==0.10.11
!pip install timm==1.0.9


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 712.0/712.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.1.1
    Uninstalling shapely-2.1.1:
      Successfully uninstalled shapely-2.1.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.8 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 1.0.15
    Uninstalling timm-1.0.15:
      Successfully uninstalled timm-1.0.15


### **3DDGD Inference:**

In [16]:
# You can download a free .obj-format example file here.
# https://www.artec3d.com/3d-models/head
# Inference
!python inference.py  \
--input_data examples  \
--model_name Mesh_MLP_MHA  \
--feature_type G0  \
--is_cropped 0

2025-06-13 05:50:33.353347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749793833.373286   18410 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749793833.379486   18410 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-13 05:50:33.400660: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

Start ...
Input Data  :  examples
Feature Type:  G0
Model Name  :  Mesh_MLP_MHA
Is Cropped  :  0
k      :  16
in_dim

## **3DDGD Inference With Gradio:**

### **Functions:**

In [7]:
%cd /content/3DDGD

/content/3DDGD


In [8]:
import os
import sys
import argparse
import numpy as np

import matplotlib.pyplot as plt
import trimesh
from trimesh import Trimesh
from glob import glob
import time
from tqdm import tqdm
import trimesh
from trimesh import Trimesh

from crop_3Dface.crop_3D_face import*
from crop_3Dface.compute_3d_landmarks import compute_landmarks_pc
from crop_3Dface.obj_handler import read_obj_file

from features_3Dmesh.features_3d_mesh import*

import torch
from torch.utils.data import Dataset, DataLoader
from pickle import load
from sklearn.preprocessing import StandardScaler

from Tabtransformer.tabtransformer import *
from Mesh_MLP_MHA.meshmlp_mha_net import *


#-------------------------------------------------------------------------------
#-------------------------------------------------------------------------------
def crop3Dface(obj_file, is_from_faceScape=0):

  mesh = np.asarray(read_obj_file(obj_file))

  # 3D facial landmarks extraction
  lm_nocrop, lm_crop = compute_landmarks_pc(
    mesh,
    img_size          = 256,
    cropping          = True,
    img_save_nocrop   = None,
    img_save_crop     = None,
    visibility_radius = 0,
    visibility_obj    = None)

  # List of landmarks
  try:
    if lm_crop == None:
      #print("len lm_nocrop : ",len(lm_nocrop))
      lm_crop = lm_nocrop.copy()
  except:
    #print("len lm_nocrop : ",len(lm_nocrop))
    #print("len lm_crop   : ",len(lm_crop))
    alpha   = 1.0

  lst_eye_l_ind  = [29,27,28,56,157,153,145,24,110,25,33]
  lst_eye_r_ind  = [286,258,257,388,390,254,253,252,256,463,414]

  # Get cropped face
  alpha   = 1.0
  mrg     = 0.0001
  if is_from_faceScape == 0:
    mrg = 0.0001
  else:
    mrg = 3.00

  cropped_face = crop_face_from_obj(obj_file, lm_crop, lst_eye_l_ind, lst_eye_r_ind, mrg, alpha, is_from_faceScape)

  # Remove irrelevant parts
  cropped_face_irr_p = remove_irrelevant_parts(cropped_face)

  # Remove eyes outliers
  vertices_face   = cropped_face_irr_p.vertices
  faces_face      = cropped_face_irr_p.faces
  vertices, faces = remove_eyes_outlier(vertices_face.tolist(), faces_face.tolist(),  lm_crop, lst_eye_l_ind, lst_eye_r_ind, 0.001)

  vertices_arr = np.array(vertices)
  faces_arr    = np.array(faces)
  cropped_face_new = Trimesh(vertices_arr, faces_arr)

  return cropped_face_new, lm_crop

#-------------------------------------------------------------------------------
class CustomInferenceDataset(Dataset):
  def __init__(self, features, scaler_path=None):

    self.features = features

    # Load the saved scaler
    if scaler_path is not None:
        self.scaler = load(open(scaler_path, "rb"))
        # Apply the scaler to the features
        self.features = self.scaler.transform(np.array(features).reshape(1, -1))

  def __len__(self):
    return 1  # Since it's a single example

  def __getitem__(self, idx):
    return torch.tensor(self.features, dtype=torch.float32)

#-------------------------------------------------------------------------------
def error_message(error_type):
  print("****************************")
  if error_type == 1:
    print("Error: The model did not load correctly. Ensure the path is correct and the model is in the specified location.")
  elif error_type == 2:
    print("Error: An error occurred during the 3D face cropping process.")
  elif error_type == 3:
    print("Error: An error occurred during the feature extraction process.")
  else:
    print("Error: An error occurred during the prediction process.")
  print("****************************")

#-------------------------------------------------------------------------------
def load_model(model_name, feature_dim):
  # Load the model
  print("Load the model ... (",model_name,")")

  if model_name == "Tabtransformer":
      try:
        if feature_dim == 624:
          model_path = "weights/Tabtransformer/Tabtransformer_G0.pth"
        if feature_dim == 2496:
          model_path = "weights/Tabtransformer/Tabtransformer_G1.pth"
        if feature_dim == 6006:
          model_path = "weights/Tabtransformer/Tabtransformer_G2.pth"

        d_model      = 128
        num_heads    = 4
        num_layers   = 3
        d_ff         = 256
        num_classes  = 1
        dropout_rate = 0.1

        model_tab = TabTransformer(feature_dim, d_model, num_heads, num_layers, d_ff, num_classes, dropout_rate)
        model_tab.load_state_dict(torch.load(model_path))
        model_tab.eval()

        return model_tab

      except:
        error_type = 1
        error_message(error_type)
        print("model_path : ",model_path)
        sys.exit(1)

  #-----------------------------------------------------------------------------
  if model_name == "Mesh_MLP_MHA":
      try:
        if feature_dim == 624:
          model_path = "weights/Mesh_MLP_MHA/Mesh_MLP_MHA_G0.pth"
        if feature_dim == 2496:
          model_path = "weights/Mesh_MLP_MHA/Mesh_MLP_MHA_G1.pth"
        if feature_dim == 6006:
          model_path = "weights/Mesh_MLP_MHA/Mesh_MLP_MHA_G2.pth"

        num_classes = 1
        drop_prob   = 0.1
        k_eig_list  = [2047, 128, 32]

        model_MHA = Net(C_in=feature_dim, C_out=num_classes, drop_path_rate=drop_prob, k_eig_list=k_eig_list)
        model_MHA.load_state_dict(torch.load(model_path))
        model_MHA.eval()

        return model_MHA

      except:
        error_type = 1
        error_message(error_type)
        print("model_path : ",model_path)
        sys.exit(1)

#-------------------------------------------------------------------------------
print("Done ...")


Done ...


In [9]:
import plotly.graph_objects as go

def plot_3d_mesh(mesh):
    # Extract vertices and faces from the mesh
    vertices = mesh.vertices
    faces    = mesh.faces

    # Create mesh3d trace for the face
    face_trace = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        colorscale=[[0, 'rgb(150, 150, 150)'], [1, 'rgb(210, 210, 210)']],
        intensity=vertices[:, 2],  # Use z-coordinate for shading
        intensitymode='vertex',
        lighting=dict(
            ambient=0.6,
            diffuse=0.5,
            fresnel=0.1,
            specular=0.2,
            roughness=0.1
        ),
        lightposition=dict(x=100, y=200, z=0),
        name='Face Mesh'
    )

    data = [face_trace]

    # Define camera for frontal z-axis view
    camera = dict(
        eye   =dict(x=0, y=0, z=2),  # Camera is positioned along the z-axis
        center=dict(x=0, y=0, z=0),  # Looking at the center of the scene
        up    =dict(x=0, y=1, z=0))  # 'Up' direction is along the y-axis

    # Create the layout with specific width and height
    layout = go.Layout(
        scene_camera=camera,
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        width =500,
        height=300
    )

    # Create the figure
    fig = go.Figure(data=data, layout=layout)

    return fig

#-------------------------------------------------------------------------------
def plot_3d_mesh_landmarks(mesh_path, landmarks):
    # Load the mesh
    mesh     = trimesh.load(mesh_path)
    vertices = mesh.vertices
    faces    = mesh.faces

    # Create mesh3d trace for the face
    face_trace = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        colorscale=[[0, 'rgb(150, 150, 150)'], [1, 'rgb(210, 210, 210)']],
        intensity=vertices[:, 2],  # Use z-coordinate for shading
        intensitymode='vertex',
        lighting=dict(
            ambient=0.6,
            diffuse=0.5,
            fresnel=0.1,
            specular=0.2,
            roughness=0.1
        ),
        lightposition=dict(x=100, y=200, z=0),
        name='Face Mesh'
    )

    # Create scatter3d trace for landmarks
    landmark_trace = go.Scatter3d(
        x=landmarks[:, 0],
        y=landmarks[:, 1],
        z=landmarks[:, 2],
        mode='markers',
        marker=dict(
            size  =3,
            color ='red',
            symbol='circle'
        ),
        name='Landmarks'
    )

    # Combine both traces
    data = [face_trace, landmark_trace]

    # Define camera for frontal z-axis view
    camera = dict(
        eye   =dict(x=0, y=0, z=2),  # Camera is positioned along the z-axis
        center=dict(x=0, y=0, z=0),  # Looking at the center of the scene
        up    =dict(x=0, y=1, z=0))  # 'Up' direction is along the y-axis

    # Create the layout with specific width and height
    layout = go.Layout(
        scene_camera=camera,
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=0),
        width =500,
        height=300
    )

    # Create the figure
    fig = go.Figure(data=data, layout=layout)

    return fig

#-------------------------------------------------------------------------------
def crop_3D_face(mesh_path, is_cropped):
  face_obj_path = mesh_path
  name_face_obj = face_obj_path.split("/")[-1]
  print(name_face_obj+" : ")

  try:
    if is_cropped == 0:
      cropped_face  = crop3Dface(face_obj_path)
      print(name_face_obj+" cropped is OK.")
      #cropped_face.export(file_obj=input_data+"/cropped_"+name_face_obj, file_type="obj")
      return cropped_face
    else:
      cropped_face = trimesh.load(face_obj_path)
      print(name_face_obj+" loaded is OK.")
      return cropped_face

  except:
    error_type = 2
    error_message(error_type)
    #sys.exit(1)

#-------------------------------------------------------------------------------
def model_predict(cropped_face, model_name, feature_type):

  device = "cuda"
  k      = 16
  in_dim = 624

  if feature_type == "G1":
    k = 64
    in_dim = 2496

  if feature_type == "G2":
    k = 154
    in_dim = 6006

  error_type = 0

  #-----------------------------------------------------------------------------
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model_loaded = load_model(model_name, in_dim)

  #-----------------------------------------------------------------------------
  if error_type == 0:
    try:
      face_mesh = normalize_faces_from_mesh(cropped_face, 15000)
      face_mesh = normalize_mesh(face_mesh)

      print("Extrinsic Features ...")
      vertex_coordinates = get_vertex_coordinates(face_mesh)
      vertex_normals     = get_vertex_normals(face_mesh)
      dihedral_angles    = generate_dihedral_angles(face_mesh)

      print("Intrinsic Features ...")
      mesh_gaussian_curvature     = generate_gaussian_curvature(face_mesh)
      eigen_vectors, eigen_values = generate_cot_eigen_vectors(face_mesh, device, 25)
      hks_features                = HKS(eigen_vectors, eigen_values, 25)

      vertex_coordinates = np.array(vertex_coordinates, copy=True)
      vertex_normals     = np.array(vertex_normals, copy=True)
      dihedral_angles    = np.array(dihedral_angles, copy=True)
      eigen_vectors      = np.array(eigen_vectors, copy=True)

      # The input features
      features = torch.cat([
          torch.from_numpy(vertex_coordinates).float(),
          torch.from_numpy(vertex_normals).float(),
          torch.from_numpy(dihedral_angles).float(),
          mesh_gaussian_curvature.float(),
          torch.from_numpy(eigen_vectors[:, 1:21]).float(),
          torch.from_numpy(hks_features).float()
      ], dim=1)

      print("features : ",features.shape)

      eigen_vectors     = eigen_vectors.astype(np.float32)  # Convert to float32 for compatibility
      features_G_tensor = torch.from_numpy(eigen_vectors)[:, :k].T.to(device) @ features.to(device)
      features_G        = torch.flatten(features_G_tensor, start_dim=0).tolist()

      print("Input features : ",len(features_G))
    except:
      error_type = 3
      error_message(error_type)
      #sys.exit(1)

    if error_type == 0:
      try:
        # Create inputs
        my_dataset = CustomInferenceDataset(features_G)
        # Create DataLoader
        inputloader = DataLoader(my_dataset, batch_size=1)

        # Use the loaded model for inference
        print("Inference ...")
        for batch in inputloader:
          with torch.no_grad():
            output    = model_loaded(batch)
            predicted = output.squeeze().item()
            if predicted >= 0.5 :
              print("Model output : is Fake. (", predicted,") \n")
            else:
              print("Model output : is Real. (", predicted,") \n")
      except:
        error_type = 4
        error_message(error_type)

  return predicted

#-------------------------------------------------------------------------------
print("Done ...")


Done ...


### **Prediction:**

In [ ]:
file_path = "/content/3DDGD/examples/man_bust.obj"
crop      = 0
cropped_face, landmarks = crop_3D_face(file_path, crop)

fig_mesh = plot_3d_mesh_landmarks(file_path, landmarks)
fig_mesh.show()


In [ ]:
# Plot cropped_face
fig_mesh_face = plot_3d_mesh(cropped_face)
fig_mesh_face.show()


In [18]:
# Prediction
labels       = ["Fake", "Real"]
model_name   = "Mesh_MLP_MHA"
feature_type = "G0"
predicted    = model_predict(cropped_face, model_name, feature_type)
print("predicted : ",predicted)


Load the model ... ( Mesh_MLP_MHA )
Extrinsic Features ...
Intrinsic Features ...
features :  torch.Size([7780, 39])
Input features :  624
Inference ...
Model output : is Real. ( 0.024476613849401474 ) 

predicted :  0.024476613849401474


### **Gradio:**

In [24]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.4.2
    Uninstalling gradio_client-1.4.2:
      Successfully uninstalled gradio_client-1.4.2


In [1]:
import gradio as gr
print(gr.__version__)

5.33.2


In [ ]:
#-------------------------------------------------------------------------------
# Run the code in the functions section again if required.
#-------------------------------------------------------------------------------

import gradio as gr

def predict_3D_deepfake(mesh_file, model_name, feature_type, is_cropped):
    crop = 1 if is_cropped == "1" else 0

    # Ensure the uploaded file path is passed correctly
    file_path = mesh_file.name  # In Google Colab, this is how you get the file path

    cropped_face, landmarks = crop_3D_face(file_path, crop)

    fig_mesh      = plot_3d_mesh_landmarks(file_path, landmarks)
    fig_mesh_face = plot_3d_mesh(cropped_face)

    # Prediction
    labels     = ["Fake", "Real"]
    predicted  = model_predict(cropped_face, model_name, feature_type)
    prediction = [predicted, 1 - predicted]
    dictionary = dict(zip(labels, map(float, prediction)))

    return fig_mesh, fig_mesh_face, dictionary


# Creating the Gradio Interface
demo = gr.Interface(
    fn=predict_3D_deepfake,
    inputs=[
        gr.File(label="Upload .obj file", file_types=[".obj"]),
        gr.Dropdown(["Tabtransformer", "Mesh_MLP_MHA"], label="Model"),
        gr.Dropdown(["G0", "G1", "G2"], label="Feature Type"),
        gr.Dropdown(["0", "1"], label="Is Cropped?"),
    ],
    outputs=[
        gr.Plot(label="3D Mesh Viewer"),
        gr.Plot(label="3D Cropped Face"),
        gr.Label(label="Result : ", num_top_classes=2),
    ],
    theme=gr.themes.Default(primary_hue="red", secondary_hue="pink"),
    title="3D Deepfake Detection",
    description="Upload a .obj :",
    article="© 2025 Hichem Felouat - hichemfel@nii.ac.jp . All rights reserved."
)

# Launch Gradio in Google Colab
demo.launch(share=True, debug=True)  #(share=True, debug=True)
